In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np
import pickle

In [2]:
# model_path='/data/model/'
model_path='model/'

In [3]:
import psutil
psutil.virtual_memory()

svmem(total=59098333184, available=56031436800, percent=5.2, used=2450894848, free=54696845312, active=2422493184, inactive=1392095232, buffers=82776064, cached=1867816960, shared=9109504, slab=213495808)

In [4]:
df_all = pd.read_csv('parsed/corpus/all.csv', index_col=0, na_filter=False)

In [5]:
with open('polish_stopwords.txt') as f:
    stop_words = [x.strip() for x in f]

tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 3), min_df=5)

In [6]:
embeddings_tfidf = tfidf_vectorizer.fit_transform(df_all.text.to_list())

In [7]:
with open(f'{model_path}embeddings-tfidf-all.pkl', "wb") as fOut:
    pickle.dump(embeddings_tfidf, fOut)

In [8]:
feature_names = np.array(tfidf_vectorizer.get_feature_names())

def get_top_tf_idf_words(response, top_n=5):
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]]

tfidf_words = [get_top_tf_idf_words(x) for x in embeddings_tfidf]

In [9]:
with open(f'{model_path}tfidf-words.pkl', "wb") as fOut:
    pickle.dump(tfidf_words, fOut)